In [104]:
try:
    import bayes_nanospace2025
    print("Already installed")
except ImportError:
    %pip install -q "bayes_nanospace2025 @ git+https://github.com/Mads-PeterVC/nanospace2025.git" # Install from GitHub. 
    print("Installed bayes_nanospace2025 from GitHub")

Already installed


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bayes_nanospace2025 import GaussianProcess, RadialBasis, Constant, Noise
import numpy as np

<div class="alert alert-block alert-danger"> <b>Warning:</b> If packages weren't already installed, please refresh this page in your <b> browser</b>, you shouldn't need to restart the Jupyter kernel, as otherwise the widgets used won't show properly. Sorry for the inconvenience! </div>

In [ ]:
def make_gp(length_scale, amplitude, noise):
    """
    Create a Gaussian Process kernel with the specified parameters.
    
    Parameters:
    - length_scale: Length scale of the radial basis function.
    - amplitude: Amplitude of the constant kernel.
    - noise: Noise level of the Gaussian Process.
    
    Returns:
    A GaussianProcess object with the specified kernel.
    """
    kernel = Constant(amplitude) * RadialBasis(length_scale=length_scale) + Noise(noise)
    return GaussianProcess(kernel=kernel, prior_mean=0.0)

In [ ]:
class KernelWidget:

    def __init__(self, X, y, X_query):
        """
        Initialize the KernelWidget with data and query points.
        
        Parameters:
        - X: Input data points.
        - y: Output values corresponding to X.
        - X_query: Points where predictions will be made.
        """
        self.X = X
        self.y = y
        self.X_query = X_query

    def visible_traces(self, fig, frame_index):
        return [trace.name in [f'frame-{frame_index}', 'data'] for trace in fig.data]
    
    def add_frame_traces(self, fig, kp, i):
        gp = make_gp(**kp)
        gp.condition(X_obs=self.X, y_obs=self.y)
        prediction = gp.predict(X_query=self.X_query)

        sigma = np.sqrt(prediction.variance)
        mean = prediction.mean
        covariance = prediction.covariance

        name = f'frame-{i}'

        # mean trace
        trace = go.Scatter(
            x=self.X_query.flatten(),
            y=mean.flatten(),
            mode='lines',
            name=name,
            line=dict(width=2),
            visible= i == 0,  # Only show the first trace initially
            showlegend=False,
        )
        fig.add_trace(trace)

        # Uncertainty fill
        trace = go.Scatter(
            x=np.concatenate([self.X_query.flatten(), self.X_query[::-1].flatten()]),
            y=np.concatenate([mean.flatten() + 2 * sigma.flatten(), (mean - 2 * sigma).flatten()[::-1]]),
            fill='toself',
            fillcolor='rgba(0, 100, 80, 0.2)',
            line=dict(color='rgba(255, 255, 255, 0)'),
            name=name,
            showlegend=False,
            visible= i == 0  # Only show the first trace initially
        )
        fig.add_trace(trace)

        # Add covariance trace in 2nd subplot
        trace = go.Heatmap(
            z=covariance.T, name=name, 
            x=self.X_query.flatten(), y=self.X_query.flatten(),
            visible= i == 0,  # Only show the first trace initially
            zmin=0, zmax=self.amp_max,
            showlegend=False,
            )
        fig.add_trace(trace, row=1, col=2)

    def add_data_traces(self, fig):
        # Add data points as scatter traces
        trace = go.Scatter(
            x=self.X.flatten(),
            y=self.y.flatten(),
            mode='markers',
            name='data',
            marker=dict(color='red', size=8),
            visible=True,
            showlegend=False
        )
        fig.add_trace(trace)

    def make_plot(self, kernel_parameters: list[dict]):

        layout = go.Layout(width=800, height=500, title="Gaussian Process Predictions",)
        fig = go.Figure(layout=layout)
        fig = make_subplots(rows=1, cols=2, figure=fig)


        self.amp_max = max(kp.get('amplitude', 1) for kp in kernel_parameters)

        self.add_data_traces(fig)

        for i, kp in enumerate(kernel_parameters):
            self.add_frame_traces(fig, kp, i)

        # Make slider
        steps = []
        for i, kp in enumerate(kernel_parameters):

            title = ""
            for key, value in kp.items():
                title += f"{key}: {value:0.3f}, "

            step = dict(
                method='update',
                args=[{'visible': self.visible_traces(fig, i)}, 
                      {'title.text': title}],
            )
            steps.append(step)

        sliders = [dict(
            active=0,
            currentvalue={'prefix': 'Kernel Parameter set: '},
            pad={'b': 10},
            steps=steps
        )]
        fig.update_layout(sliders=sliders)
        fig.update_layout(
            yaxis=dict(title='Output (y)', range=[-2, 2]),

        )

        fig.show()

        return fig


In [105]:
# You can change this if you want to use different data.

# Define some observations
X = np.array([0.3, 0.5, 0.7]).reshape(-1, 1)
y = np.array([-0.5, 0.5, 0]).reshape(-1, 1)
X_query = np.linspace(0, 1, 250).reshape(-1, 1) # Points where we want to make predictions

### Vary the length-scale

In [ ]:
kw = KernelWidget(X, y, X_query)

kernel_parameters = [
    {'length_scale': ls, 'amplitude': 1.0, 'noise': 0.001} for ls in np.linspace(0.01, 1.0, 25)
]

fig = kw.make_plot(kernel_parameters=kernel_parameters)

### Vary the noise

In [ ]:
kw = KernelWidget(X, y, X_query)

kernel_parameters = [
    {'length_scale': 0.2, 'amplitude': 1.0, 'noise': noise} for noise in np.geomspace(1e-3, 1.0, 25)]

fig = kw.make_plot(kernel_parameters=kernel_parameters)

### Vary amplitude

In [ ]:
kw = KernelWidget(X, y, X_query)

kernel_parameters = [
    {'length_scale': 0.2, 'amplitude': amplitude, 'noise': 1e-3} for amplitude in np.linspace(0.01, 2.0, 30)]

fig = kw.make_plot(kernel_parameters=kernel_parameters)

### Vary length-scale and noise

In [ ]:
kw = KernelWidget(X, y, X_query)

kernel_parameters = [
    {'length_scale': ls, 'amplitude': 1.0, 'noise': noise} for ls, noise in zip(np.linspace(0.01, 0.5, 30), np.geomspace(0.01, 2.0, 30))]

fig = kw.make_plot(kernel_parameters=kernel_parameters)